In [126]:
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch

In [127]:
csv_path = r"C:\Users\14282\OneDrive\桌面\fasion_mnist\fashion-mnist_train.csv"
data = pd.read_csv(csv_path)

In [128]:
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [129]:
label = data.iloc[:, 0].values
label

array([2, 9, 6, ..., 8, 8, 7], dtype=int64)

In [130]:
label.shape

(60000,)

In [131]:
train_set = data.iloc[:, 1:].values
train_set

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [132]:
train_set.shape

(60000, 784)

In [133]:
train_set = train_set.astype(np.uint8)
train_set

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [134]:
train_set = train_set.reshape(60000, 28, 28)
train_set.shape

(60000, 28, 28)

In [135]:
train_set[0].shape

(28, 28)

In [136]:
target = int(label[0])
target

2

In [137]:
transform = transforms.Compose([transforms.ToTensor()])

In [138]:
img = transform(train_set)
img.shape
#有问题

torch.Size([28, 60000, 28])

In [139]:
len(train_set)

60000

In [140]:
class Fasion_Mnist(Dataset):
    def __init__(self, csv_path, transform=None):
        data = pd.read_csv(csv_path)       
        label = data.iloc[:, 0].values
        train_set = data.iloc[:, 1:].values.astype(np.uint8).reshape(len(label), 28, 28)
        # train_set /= 255.0 
       
        self.train_set = train_set
        self.train_label = label
        self.transform = transform
    
    def __getitem__(self, index):
        img = self.train_set[index]
        target = int(self.train_label[index])
        #为什么要转换成int呢？
        if self.transform is not None:
            img = self.transform(img)
        return img, target
    def __len__(self):
        return len(self.train_set)

train_dataset = Fasion_Mnist(r"C:\Users\14282\OneDrive\桌面\fasion_mnist\fashion-mnist_train.csv", transform=transform)
test_dataset = Fasion_Mnist(r"C:\Users\14282\OneDrive\桌面\fasion_mnist\fashion-mnist_test.csv", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [141]:
train_dataset

In [142]:
train_loader

In [143]:
for batch_idx, data in enumerate(train_loader, 0):
    inputs, target = data

In [144]:
inputs.shape, target.shape
#为啥是32呢？

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

In [147]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=3),
            torch.nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, kernel_size=3),
            torch.nn.MaxPool2d(2,2),
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(400, 120),
            torch.nn.Linear(120, 84),
            torch.nn.Linear(84, 10)
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = self.conv1(x)  
        x = self.conv2(x)  
        x = x.view(batch_size, -1) 
        x = self.fc(x)
        return x  

model = Net()

In [153]:
outputs = model(inputs)
outputs.shape

torch.Size([32, 10])

In [156]:
outputs.data[0]
#detach()方法

tensor([ 0.0746, -0.0591, -0.0528, -0.0737, -0.0916, -0.1144, -0.0981,  0.0041,
        -0.0929, -0.0611])

In [163]:
_, predicted = torch.max(outputs.data, dim=1)
predicted
#有点问题

tensor([0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 7, 0, 0, 0, 0, 0])